In [1]:
pip install nltk Sastrawi pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import re, string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [3]:
df = pd.read_csv("whatsapp_reviews.csv")

In [4]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,e1050486-a412-4827-b3c0-de2a20a19ad4,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,KA peget wa long be jeut edit na lagu kedro,5,0,2.24.4.76,2025-06-07 16:30:15,NaN,NaN,2.24.4.76
1,51d1e54f-8cc8-4f3b-9813-bf5d0eb6e8d2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"naha nya bug na aneh, padahal ga di pivasi sw ...",1,0,NaN,2025-06-07 16:29:56,NaN,NaN,NaN
2,650cdc3e-37e2-4f32-8a9d-51a3f4a2ba5a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sasadusasadu,2,0,2.25.16.84,2025-06-07 16:29:30,NaN,NaN,2.25.16.84
3,5fbbbae9-0d50-4baf-839b-87f564486ed5,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,tidak menerima kode,5,0,2.25.16.84,2025-06-07 16:28:45,NaN,NaN,2.25.16.84
4,950ed213-80b2-43b2-a138-d9309ce0cae0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,saya suka,5,0,2.25.16.84,2025-06-07 16:27:57,NaN,NaN,2.25.16.84


In [5]:
df.columns

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion'],
      dtype='object')

In [6]:
def clean_text(text):
    # Hapus emoji
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # Lowercase
    text = text.lower()
    
    # Hapus URL
    text = re.sub(r'http\S+|www.\S+', '', text)
    
    # Hapus angka
    text = re.sub(r'\d+', '', text)
    
    # Hapus tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [7]:
df['cleaned'] = df['content'].astype(str).apply(clean_text)
df[['content', 'cleaned']].head()

,content,cleaned
0,KA peget wa long be jeut edit na lagu kedro,ka peget wa long be jeut edit na lagu kedro
1,"naha nya bug na aneh, padahal ga di pivasi sw ...",naha nya bug na aneh padahal ga di pivasi sw t...
2,sasadusasadu,sasadusasadu
3,tidak menerima kode,tidak menerima kode
4,saya suka,saya suka


In [8]:
# Daftar kata kunci sentimen
positive_keywords = ['bagus', 'baik', 'mantap', 'suka', 'puas', 'keren', 'cepat', 'mudah', 'memuaskan']
negative_keywords = ['jelek', 'buruk', 'parah', 'tidak suka', 'kecewa', 'lemot', 'error', 'macet', 'lambat']

# Fungsi pelabelan berdasarkan keyword
def label_sentiment(text):
    for word in positive_keywords:
        if word in text:
            return 'positif'
    for word in negative_keywords:
        if word in text:
            return 'negatif'
    return 'netral'  # fallback default

# Terapkan ke kolom 'cleaned'
df['sentiment'] = df['cleaned'].apply(label_sentiment)

# Cek hasil awal
df[['content', 'cleaned', 'sentiment']].head()

,content,cleaned,sentiment
0,KA peget wa long be jeut edit na lagu kedro,ka peget wa long be jeut edit na lagu kedro,netral
1,"naha nya bug na aneh, padahal ga di pivasi sw ...",naha nya bug na aneh padahal ga di pivasi sw t...,netral
2,sasadusasadu,sasadusasadu,netral
3,tidak menerima kode,tidak menerima kode,netral
4,saya suka,saya suka,positif


In [9]:
df.to_csv("whatsapp_preprocessed_labeled.csv", index=False)